In [1]:
import pprint
from tatsu import parse
from tabulate import tabulate
from IPython.display import HTML, display

In [2]:
GRAMMAR = '''
    start = expression $ ;
    
    trash = /\w+/;

    number 
        = /\d+/ 
        ;
        
    factor
        = number
        ;

    expression
        =
        | expression expOperator term
        | trash term
        | trash expression
        ;
    
    term
        =
        | term termOperator factor
        | factor
        ;

    expOperator
        =
        | plusOperator
        | minusOperator
        | trash expOperator
        ;
        
    termOperator
        =
        | multiplicationOperator
        | divisionOperator
        | trash termOperator
        ;
        
    plusOperator
        =
        | 'plus'
        | 'dodać'
        ;
    
    minusOperator
        =
        | 'minus'
        | 'odjąć'
        ;
        
    multiplicationOperator
        = 'razy'
        ;
    
    divisionOperator
        =
        | 'przez'
        | 'dzielone przez'
        | 'nad'
        ;
        
'''

In [3]:
operatorMap = {
    "plus": "+",
    "dodać": "+",
    "minus": "-",
    "odjąć": "-",
    "razy": "*",
    "przez": "/",
    "dzielone przez": "/"
}

In [4]:
def process(e):
    if e in ['+', '-', '*', '/']:
        return e
    if e.isdigit():
        return e
    if e in operatorMap.keys():
        return operatorMap.get(e)
    else:
        return ''

In [5]:
def debugEvaluate(e):
    result = ''
    print('evaluate:', e)
    for subexpression in e:
        if type(subexpression) is not str:
            partial = debugEvaluate(subexpression)
            result += partial
        else:
            result += process(subexpression)
    print('result:', result)
    return result

def evaluate(e):
    result = ''
    for subexpression in e:
        if type(subexpression) is not str:
            partial = evaluate(subexpression)
            result += partial
        else:
            result += process(subexpression)
    return result

In [6]:
expression = 'policz proszę 5 razy 3 i dodać 88 razy 2'
parsedExpression = parse(GRAMMAR, expression)
pprint.pprint(parsedExpression, indent=2, width=20)

[ 'policz',
  [ 'proszę',
    [ [ '5',
        'razy',
        '3'],
      [ 'i',
        [ 'dodać',
          [ '88',
            'razy']]],
      '2']]]


In [7]:
evaluatedExpression = debugEvaluate(parsedExpression)

evaluate: ['policz', ['proszę', [['5', 'razy', '3'], ['i', ['dodać', ['88', 'razy']]], '2']]]
evaluate: ['proszę', [['5', 'razy', '3'], ['i', ['dodać', ['88', 'razy']]], '2']]
evaluate: [['5', 'razy', '3'], ['i', ['dodać', ['88', 'razy']]], '2']
evaluate: ['5', 'razy', '3']
result: 5*3
evaluate: ['i', ['dodać', ['88', 'razy']]]
evaluate: ['dodać', ['88', 'razy']]
evaluate: ['88', 'razy']
result: 88*
result: +88*
result: +88*
result: 5*3+88*2
result: 5*3+88*2
result: 5*3+88*2


In [8]:
print(evaluatedExpression)

5*3+88*2


In [9]:
eval(evaluatedExpression)

191

In [10]:
def calculate(expression):
    parsedExpression = parse(GRAMMAR, expression)
    evaluatedExpression = evaluate(parsedExpression)
    result = eval(evaluatedExpression)
    return result

In [11]:
testSentences = [
    'policz proszę 5 razy 3 i dodać 88 razy 2',
    'ile jest 2 plus 2 razy 2',
    'ile jest 10 plus 6 przez 2',
    '2 razy 3',
    '2 plus 2 dodać 2',
    '2 plus 2',
    '10 przez 2',
    'weź mnie policz 3 razy 6 przez 2 dodać 8 razy 5'
]

In [12]:
result = [[sentence, calculate(sentence)] for sentence in testSentences]

In [13]:
display(HTML(tabulate(result, tablefmt='html', headers=["Sentence", "Evaluation"])))

Sentence,Evaluation
policz proszę 5 razy 3 i dodać 88 razy 2,191
ile jest 2 plus 2 razy 2,6
ile jest 10 plus 6 przez 2,13
2 razy 3,6
2 plus 2 dodać 2,6
2 plus 2,4
10 przez 2,5
weź mnie policz 3 razy 6 przez 2 dodać 8 razy 5,49
